In [ ]:
from numpy.random import seed
seed(1)
import tensorflow

tensorflow.random.set_seed(2)

In [ ]:
import sys
import os
from datetime import datetime
import numpy as np
import pandas as pd
import json
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle as cp
from tensorflow import keras
import tensorflow as tf
from keras import backend as K
import datetime as dt
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

###  Hyperparameters

In [ ]:
hidden_units = 200
epochs = 100
lookback = 7
learning_rate = 0.001
batch_size = 1024
regularizer = 'l2'
dropouts = {'dropout':0.55,'recurrent_dropout':0.3}

crops = ['Onion','Potato','Wheat','Rice']
crop = crops[0]

others = True

diff = False

train_split = 2557 #- 1 #if using price diff
seasons = [12,0]#,365]
normalizations = [True]
epochs = [100]
hidden_units = [200]

fileformat = 'food_price_data/%s.csv'

In [ ]:
def toPDTimeSeries(crop,start,end):
    startdate = dt.datetime.strptime(start, '%Y-%m-%d').date()
    enddate = dt.datetime.strptime(end, '%Y-%m-%d').date()
    data = pd.read_csv(fileformat%(crop))
    print(data.head())
    price = data['price']
    dates = data['date']
    
    dateindex = pd.DatetimeIndex(dates)
    print(dateindex[0]+31)
    print (dateindex)
    price_data = pd.Series(price)#, index=dates)#index)
    return price_data[start:end]
    

In [ ]:
def addSpanLag(data, events=[], look_back=7, look_ahead = 7, season=0):##span==1
#     ### Adding span (required for weekly, monthly average data)
#     if span>1:
#         nts = []
#         ind = 0
#         while ind < len(ts):
#             tmpts = ts[ind:ind+span]
#             print (tmpts)
#             dt = tmpts[0]
#             vals = [k for k in tmpts]
#             pr = sum(vals)/len(vals)
#             nts.append((dt,pr))
#             ind+=span
#         ts = nts
#     if not convert_to_Xy:
#         return ts

    ### Adding lag (based on look back and look ahead)
    
    if len(events) > 0:
        input = np.append(data,events,axis=1)
        if season>0:
            if season==12:
                season = np.array([k.date().month for k in pd.date_range(start='1/1/2006', end='12/31/2020')])
            elif season==365:
                season = np.array([k%season for k in range(data.shape[0])])
                
            season = season.reshape(len(season),1)
            input = np.append(input,season,axis=1)
        
    else:
        input = data
        
        
    ind = 0
    X_data = []
    y_data = []

    while ind + look_back + look_ahead < len(data):
        
        X = input[ind:ind+look_back]
        Y = data[ind+look_back+look_ahead]
        X_data.append(X)
        y_data.append(Y)
        ind+=1
    X_data = np.array(X_data)
    y_data = np.array(y_data)

    return X_data, y_data

### Training

In [ ]:


def ren_train(X, y, hidden_units=100, learning_rate=0.01, epochs=200, batch_size=256,dropouts={'dropout':0.25,'recurrent_dropout':0.25},regularizer='l2'):
    model = Sequential()
    model.add(LSTM(200, activation='relu', recurrent_regularizer='l2', activity_regularizer='l2', 
                   dropout=0.35, recurrent_dropout=0.25, 
                   input_shape=(X.shape[1], X.shape[-1])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(X, y, epochs=100, verbose=1)
    plt.plot(history.history['loss'])
    plt.show()
    return model

In [ ]:
i=1
for epoch in epochs:
    for hidden_unit in hidden_units:
        for season in seasons:
            for normalization in normalizations:
                model_suffix = "crop=%s_norm=%s_season=%s"%(crop,normalization,season)
                print(str(i),model_suffix)
                i+=1
                
                food_price = pd.read_csv(fileformat%(crop))['price']
                if diff:
                    food_price = food_price.diff().dropna() 
                food_price = np.array(food_price).reshape(food_price.shape[0],1)

                mean = np.mean(food_price)
                std = np.std(food_price)
                if normalization:
                    mean = np.mean(food_price)
                    std = np.std(food_price)
                    food_price = (food_price - mean)/std

                ### Events
                events_train = np.load('embeddings/events_train.npy')
                events_test = np.load('embeddings/events_test.npy')
                events = np.append(events_train,events_test, axis=0)

                ### Topics
                topics_train = np.load('embeddings/toi.maxLDA_train.npy')
                topic_test = np.load('embeddings/toi.maxLDA_test.npy')
                topics = np.append(events_train,events_test, axis=0)

                ### Word2Vec
                w2v_train = np.load('embeddings/toi.W2V_train.npy')
                w2v_test = np.load('embeddings/toi.W2V_test.npy')
                w2v = np.append(events_train,events_test, axis=0)

                X_event, y_event   = addSpanLag(food_price,events,look_back=7,look_ahead=6,season=season)
                X_event_train = X_event[:train_split]
                y_event_train = y_event[:train_split]
                X_event_test = X_event[train_split:]
                y_event_test = y_event[train_split:]

                X_topics, y_topics   = addSpanLag(food_price,topics,look_back=7,look_ahead=6,season=season)
                X_topics_train = X_topics[:train_split]
                y_topics_train = y_topics[:train_split]
                X_topics_test = X_topics[train_split:]
                y_topics_test = y_topics[train_split:]

                X_w2v, y_w2v   = addSpanLag(food_price,w2v,look_back=7,look_ahead=6,season=season)
                X_w2v_train = X_w2v[:train_split]
                y_w2v_train = y_w2v[:train_split]
                X_w2v_test = X_w2v[train_split:]
                y_w2v_test = y_w2v[train_split:]


                X_noevent, y_noevent   = addSpanLag(food_price,look_back=7,look_ahead=6,season=season)
                X_noevent_train = X_noevent[:train_split]
                y_noevent_train = y_noevent[:train_split]
                X_noevent_test = X_noevent[train_split:]
                y_noevent_test = y_noevent[train_split:]


                ### REN
                model_event = ren_train(X_event_train,y_event_train)
                
                ### LSTM
                model_noevent = ren_train(X_noevent_train,y_noevent_train)
                

                model_event.save('ren'+model_suffix)
                model_noevent.save('lstm'+model_suffix)



                if others:
                    ### LDA
                    model_topics = ren_train(X_topics_train,y_topics_train)

                    ### LSTM
                    model_w2v = ren_train(X_w2v_train,y_w2v_train)

                    model_topics.save('lda'+model_suffix)
                    model_w2v.save('w2v'+model_suffix)

